In [ ]:
from helper import *

import pandas as pd
import numpy as np
import os
import csv
import sys
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 8]
# use LaTeX fonts in the plot
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
%matplotlib inline

In [ ]:
OUTPUT = "level"
EVAL_DIR = "/home/felix/todo/algohol/level"
MLP_METHODS = ["merge"]
MLP_LEVELS = [[int(2 ** i)] for i in np.arange(9.0, 11.5, 1.0)]
MLP_LEVELS = MLP_LEVELS + [[int(2 ** i), 4] for i in np.arange(9.0, 11.5, 1.0)]
MLP_LEVELS = MLP_LEVELS + [[int(2 ** i), 16] for i in np.arange(9.0, 11.5, 1.0)]
MLP_LEVELS = MLP_LEVELS + [[int(2 ** i), 4, 4] for i in np.arange(9.0, 11.5, 1.0)]
FAST_QUERY_METHODS = ["pcrp", "pch", "prp"]
QUERY_METHODS = ["normal"] + FAST_QUERY_METHODS
AREAS = ["baden-wuerttemberg"]
SKIP_COUNTS = True
print(MLP_METHODS, "with", MLP_LEVELS)

# Time

In [ ]:
df_query = pd.DataFrame()
for area in AREAS:
    for query in QUERY_METHODS:
        if query == "normal" or query == "bi":
            tmp = pd.read_json(EVAL_DIR + "/" + area + "-" + query + "-time.json")
            tmp["Area"] = area
            tmp["Query"] = query
            df_query = df_query.append(tmp, ignore_index = True)
        else:
            for mlp in MLP_METHODS:
                for partitions in MLP_LEVELS:
                    tmp = pd.read_json(EVAL_DIR + "/" + area + "-" + mlp + "-" + "_".join(map(str, partitions)) + "-" + query + "-time.json")
                    tmp["Area"] = area
                    tmp["Query"] = query
                    tmp["Mlp"] = mlp
                    tmp["Mlp_partitions"] = "_".join(map(str, partitions))
                    df_query = df_query.append(tmp, ignore_index = True)

In [ ]:
with open(EVAL_DIR + "/log.json") as file:
    tmp = file.readlines()
    log = pd.read_json(" ".join(tmp), orient='index')
# create command string to not match against list
log['command_string'] = log['command'].agg(lambda x: ' '.join(map(str, x)))

In [ ]:
speedups = list()
for area in AREAS:
    dijkstra = df_query[(df_query.Query == "normal") & (df_query.Area == area)]["time"].mean()
    bidijkstra = df_query[(df_query.Query == "bi") & (df_query.Area == area)]["time"].mean()
    for mlp in MLP_METHODS:
        for partitions in MLP_LEVELS:
            row = dict()
            for query in FAST_QUERY_METHODS:
                tmp = df_query[(df_query.Area == area) & (df_query.Query == query) & (df_query.Mlp == mlp) & (df_query.Mlp_partitions == "_".join(map(str, partitions)))]
                row[query.upper() + "-time"] = ns_to_ms(tmp["time"].mean())
                row[query.upper() + "-speedup"] = dijkstra / tmp["time"].mean()
            
            lines = log[(log['command_string'].str.contains("prp_pre")) & (log['command_string'].str.contains(mlp + "-" + "_".join(map(str, partitions)) + ".bin"))]
            if lines.shape[0] == 0:
                continue
            line = lines.iloc[-1]
            runtime = int(sec_to_min(float(line['time'].replace(" seconds", ""))))
            
            pindex = {"partitions": "_".join(map(str, partitions)), "pre [min]": runtime}
            speedups.append({**pindex, **row})

In [ ]:
def special_hacky_sort(x):
    splited = x.str.split("-", expand=True)
    return pd.DataFrame(splited).astype(float).sum(axis=1)

In [ ]:
df_speedups = pd.DataFrame(speedups)
df_speedups['partitions'] = df_speedups['partitions'].str.replace('_','-')
assert((df_speedups.groupby(["partitions", "pre [min]"]).count().all() == 1).iloc[0])
df_speedups = df_speedups.groupby(["partitions", "pre [min]"]).first()
df_speedups = df_speedups.sort_values(by="partitions", key=special_hacky_sort)
latex = df_speedups.to_latex(float_format="{:0.1f}".format)
df_speedups

In [ ]:
# fixup ugly latex code to have single line header
latex_list = latex.splitlines()

columns = latex_list[2].split("&")
indices = latex_list[3].split("&")

latex_list[2] = ""
for query in FAST_QUERY_METHODS:
    latex_list[2] += "& \multicolumn{2}{c|}{" + query.upper() + "} "
latex_list[2] += " \\\\"
# remove last pipe from "c"
latex_list[2] = "c".join(latex_list[2].rsplit("c|", 1))

latex_list[3] = "&".join(indices[:1] + columns[1:])

for query in FAST_QUERY_METHODS:
    latex_list[3] = latex_list[3].replace(query.upper() + "-speedup", "speedup").replace(query.upper() + "-time", "[ms]")

latex_list[0] = latex_list[0].replace('lr', 'l|r', 1)
latex_list[0] = latex_list[0].replace('rr', 'rr|', len(FAST_QUERY_METHODS) - 1)

latex_list.insert(len(latex_list)-10, '\midrule')
latex_list.insert(len(latex_list)-6, '\midrule')

latex_new = '\n'.join(latex_list)
with open(OUTPUT + "-speedups.tex", "w") as latex_file:
    latex_file.writelines(latex_new)